#### review
![avatar](./imgs/img_1.png)

#### trainer arguments & trainer

In [2]:
from transformers import Trainer, TrainingArguments
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import wandb

In [3]:
class DistillTrainingArguments(TrainingArguments):
    # TrainingArguments: @dataclass
    # 增加两个 KD 所需的参数参数
    def __init__(self, *args, alpha=0.5, temperature=2., **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

In [5]:
class DistillTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs, return_outputs=False):
        s_output = model(**inputs)
        s_ce = s_output.loss
        s_logits = s_output.logits

        with torch.no_grad():
            t_output = self.teacher_model(**inputs)
            t_logits = t_output.logits

        loss_kl_fct = nn.KLDivLoss(reduction="batchmean")
        loss_kd = self.args.temperature ** 2 * loss_kl_fct(
            F.log_softmax(s_logits / self.args.temperature, dim=-1),
            F.softmax(t_logits / self.args.temperature, dim=-1),
        )

        loss = self.args.alpha * s_ce + (1 - self.args.alpha) * loss_kd
        return (loss, s_output) if return_outputs else loss

#### pipeline
##### datasets

In [9]:
import os
os.environ['HTTP_PROXY'] = 'http://192.168.0.103:2022'
os.environ['HTTPS_PROXY'] = 'http://192.168.0.103:2022'

In [10]:
from datasets import load_dataset
clinc = load_dataset("clinc_oos", "plus")

Using the latest cached version of the module from /home/zhuyuedlut/.cache/huggingface/modules/datasets_modules/datasets/clinc_oos/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1 (last modified on Wed Jul 12 21:58:36 2023) since it couldn't be found locally at clinc_oos., or remotely on the Hugging Face Hub.


HF google storage unreachable. Downloading and preparing it from source


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 from datasets import load_dataset                                                            │
│ ❱ 2 clinc = load_dataset("clinc_oos", "plus")                                                    │
│   3                                                                                              │
│                                                                                                  │
│ /home/zhuyuedlut/anaconda3/envs/llm/lib/python3.10/site-packages/datasets/load.py:1797 in        │
│ load_dataset                                                                                     │
│                                                                                                  │
│   1794 │   try_from_hf_gcs = path not in _PACKAGED_DATASETS_MODULES                              │
│   1795 │                                                                                         │
│   1796 │   # Download and prepare data                                                           │
│ ❱ 1797 │   builder_instance.download_and_prepare(                                                │
│   1798 │   │   download_config=download_config,                                                  │
│   1799 │   │   download_mode=download_mode,                                                      │
│   1800 │   │   verification_mode=verification_mode,                                              │
│                                                                                                  │
│ /home/zhuyuedlut/anaconda3/envs/llm/lib/python3.10/site-packages/datasets/builder.py:890 in      │
│ download_and_prepare                                                                             │
│                                                                                                  │
│    887 │   │   │   │   │   │   │   prepare_split_kwargs["max_shard_size"] = max_shard_size       │
│    888 │   │   │   │   │   │   if num_proc is not None:                                          │
│    889 │   │   │   │   │   │   │   prepare_split_kwargs["num_proc"] = num_proc                   │
│ ❱  890 │   │   │   │   │   │   self._download_and_prepare(                                       │
│    891 │   │   │   │   │   │   │   dl_manager=dl_manager,                                        │
│    892 │   │   │   │   │   │   │   verification_mode=verification_mode,                          │
│    893 │   │   │   │   │   │   │   **prepare_split_kwargs,                                       │
│                                                                                                  │
│ /home/zhuyuedlut/anaconda3/envs/llm/lib/python3.10/site-packages/datasets/builder.py:1649 in     │
│ _download_and_prepare                                                                            │
│                                                                                                  │
│   1646 │   │   yield job_id, True, (total_num_examples, total_num_bytes, writer._features, num_  │
│   1647 │                                                                                         │
│   1648 │   def _download_and_prepare(self, dl_manager, verification_mode, **prepare_splits_kwar  │
│ ❱ 1649 │   │   super()._download_and_prepare(                                                    │
│   1650 │   │   │   dl_manager,                                                                   │
│   1651 │   │   │   verification_mode,                                                            │
│   1652 │   │   │   check_duplicate_keys=verification_mode == VerificationMode.BASIC_CHECKS       │
│                                                            

In [ ]:
clinc

In [ ]:
clinc['train'][:10]

In [ ]:
intents = clinc['train'].features['intent']
num_labels = intents.num_classes
num_labels

#### Student model初始化